# <font color='cyan'>Chatbot API with Keras Model</font>

### Main challenges:
- Classify user input to recognise intent
- keep context

In [1]:
# Keras deep learning library to build classification model
# Lancaster stemming library used to collapse distinct word forms
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

Using TensorFlow backend.


In [ ]:
'''
Chatbot intents & patterns to learn defined in a plain JSON file
Classification model can be created for small vocabulary
Need to build vocabulary > patterns processed
each word stemmed to produce generic root > help cover more combi for inputs
'''

### Read the JSON file

In [7]:
import json

with open('intents.json') as intent_file:
    intents = json.load(intent_file)

In [8]:
words = []
classes = []
documents = []
ignore_words = ['?']

#loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in sentence
        w = nltk.word_tokenize(pattern)
        
        # add to words list
        words.extend(w)
        
        # add documents in corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem & lower ea word & remove dupes
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns & intents
print(len(documents), 'documents')
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocab
print(len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [ ]:
'''
Training won't be run based on vocab of words (words meaningless for machine)
Need to translate words into bags of words with arr containing 0/1

Arr length will be equal to vocab size & 1 set when word from current
pattern is located in a given position
'''

In [12]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [13]:
# training set, bag of words for ea sentence
for doc in documents:
    # initialise our bag of words
    bag = []
    #list tokenized words for the pattern
    pattern_words = doc[0]
    # stem ea word - create base word to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words arr with 1, if word match found in
    # current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # output is '0' for ea tag & '1' for current tag (for ea pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [14]:
# shuffle our features & turn into np.array
random.shuffle(training)
training = np.array(training)

In [15]:
# create train & test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
'''
Training data - X (pattern converted into array [0,1,0,1...,0])
Y (intents converted into arr [1,0,0,0,..,0]), will be single 1 for intents arr

Model built on 3 layers

Classification output will be multiclass arr > help identify encoded intent

Use Softmax activation to produce multiclass classification output
(result returns arr of 0/1: [1,0,0...,0] - identifies encoded intent)
'''

### Compile Keras model with SGD optimiser

In [23]:
# Compile model. Stochastic gradient descent with Nesterov accelerated
# gradient gives good results for this model

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd,
              metrics = ['accuracy'])

### Fit model

In [24]:
# Execute training & construct classification model

In [25]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs = 200,
          batch_size = 5, verbose = 1)

Epoch 1/200
45/45 [==============================] - 0s 2ms/step - loss: 2.2496 - accuracy: 0.0444
Epoch 2/200
45/45 [==============================] - 0s 260us/step - loss: 2.1295 - accuracy: 0.2000
Epoch 3/200
45/45 [==============================] - 0s 229us/step - loss: 2.1159 - accuracy: 0.3111
Epoch 4/200
45/45 [==============================] - 0s 227us/step - loss: 2.0337 - accuracy: 0.2667
Epoch 5/200
45/45 [==============================] - 0s 247us/step - loss: 1.9254 - accuracy: 0.4000
Epoch 6/200
45/45 [==============================] - 0s 271us/step - loss: 1.8528 - accuracy: 0.3556
Epoch 7/200
45/45 [==============================] - 0s 207us/step - loss: 1.7022 - accuracy: 0.4000
Epoch 8/200
45/45 [==============================] - 0s 248us/step - loss: 1.5451 - accuracy: 0.6222
Epoch 9/200
45/45 [==============================] - 0s 257us/step - loss: 1.4402 - accuracy: 0.6889
Epoch 10/200
45/45 [==============================] - 0s 265us/step - loss: 1.2631 - accuracy

45/45 [==============================] - 0s 199us/step - loss: 0.0541 - accuracy: 1.0000
Epoch 82/200
45/45 [==============================] - 0s 230us/step - loss: 0.0644 - accuracy: 0.9778
Epoch 83/200
45/45 [==============================] - 0s 186us/step - loss: 0.1286 - accuracy: 0.9333
Epoch 84/200
45/45 [==============================] - 0s 221us/step - loss: 0.0572 - accuracy: 1.0000
Epoch 85/200
45/45 [==============================] - 0s 220us/step - loss: 0.0383 - accuracy: 1.0000
Epoch 86/200
45/45 [==============================] - 0s 192us/step - loss: 0.0491 - accuracy: 1.0000
Epoch 87/200
45/45 [==============================] - 0s 218us/step - loss: 0.0285 - accuracy: 1.0000
Epoch 88/200
45/45 [==============================] - 0s 200us/step - loss: 0.0634 - accuracy: 0.9778
Epoch 89/200
45/45 [==============================] - 0s 180us/step - loss: 0.0561 - accuracy: 0.9778
Epoch 90/200
45/45 [==============================] - 0s 187us/step - loss: 0.0457 - accuracy: 

45/45 [==============================] - 0s 155us/step - loss: 0.0156 - accuracy: 1.0000
Epoch 161/200
45/45 [==============================] - 0s 189us/step - loss: 0.0137 - accuracy: 1.0000
Epoch 162/200
45/45 [==============================] - 0s 175us/step - loss: 0.0081 - accuracy: 1.0000
Epoch 163/200
45/45 [==============================] - 0s 255us/step - loss: 0.0215 - accuracy: 1.0000
Epoch 164/200
45/45 [==============================] - 0s 179us/step - loss: 0.0343 - accuracy: 1.0000
Epoch 165/200
45/45 [==============================] - 0s 181us/step - loss: 0.0216 - accuracy: 1.0000
Epoch 166/200
45/45 [==============================] - 0s 181us/step - loss: 0.0237 - accuracy: 1.0000
Epoch 167/200
45/45 [==============================] - 0s 199us/step - loss: 0.0077 - accuracy: 1.0000
Epoch 168/200
45/45 [==============================] - 0s 178us/step - loss: 0.0380 - accuracy: 0.9778
Epoch 169/200
45/45 [==============================] - 0s 165us/step - loss: 0.0179 - a

### Define helper functions

In [26]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into arr
    sentence_words = nltk.word_tokenize(sentence)
    # stem ea word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [27]:
# Translate user sentence into bag of words with arr 0/1
# 0 or 1 for ea word in bag that exists in the sentence
def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocab matrix
    bag = [0] * len(words)
    
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word in vocab position
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    
    return np.array(bag)

### Example translating sentence into bag of words

In [28]:
p = bow("Load blood pressure for patient", words)
print(p)
print(classes)

Found in bag: load
Found in bag: blood
Found in bag: press
Found in bag: for
Found in bag: paty
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [ ]:
'''
Good practice to save trained model into pickle file to reuse to publish
through Flask REST API
'''

In [29]:
# Use pickle to load in pre-trained model
global graph
graph = tf.get_default_graph()

with open(f"katana-assistant-model.pkl", 'rb') as f:
    model = pickle.load(f)

NameError: name 'tf' is not defined

In [ ]:
'''
before publishing model through Flask REST API, always good to run extra test
use model.predict to classify user input & based on calculated
probability return intent (multiple intents can be returned)
'''

In [30]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from model
    input_data = pd.DataFrame([bow(sentence, words)], dtype = float,
                              index = ['input'])
    results = model.predict([input_data])[0]
    # filter predictions below a threshold & provide intent index
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key = lambda x: x[1], reverse = True)
    
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent & probability
    return return_list

In [31]:
classify_local('Fetch blood result for patient')

Found in bag: blood
Found in bag: result
Found in bag: for
Found in bag: paty


[('blood_pressure_search', '0.9999262')]

### Publish same function through REST endpoint